In [1]:
import polars as pl
import polars.selectors as cs

import altair as alt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as dt

# Feature Enginiering

In [2]:
spotify_history_path = r"D:\Datasets\Data Preaparations\spotify_history.csv"
spotify_data_dictionary_description_path = r"D:\Datasets\Data Preaparations\spotify_data_dictionary Description.csv"

In [3]:
spotify_history_df = pl.read_csv(spotify_history_path)

In [4]:
spotify_data_dictionary_description_df = pl.read_csv(spotify_data_dictionary_description_path)

In [6]:
spotify_history_df.describe()

statistic,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped
str,str,str,str,f64,str,str,str,str,str,f64,f64
"""count""","""149860""","""149860""","""149860""",149860.0,"""149860""","""149860""","""149860""","""149717""","""149743""",149860.0,149860.0
"""null_count""","""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""143""","""117""",0.0,0.0
"""mean""",null,null,null,128316.635093,null,null,null,null,null,0.744582,0.052509
"""std""",null,null,null,117840.060332,null,null,null,null,null,null,null
"""min""","""003d3VbyJTZiiOYT2W7fnQ""","""2013-07-08 02:44:34""","""android""",0.0,"""!!!!!!!""","""""Weird Al"" Yankovic""","""!!Going Places!!""","""appload""","""appload""",0.0,0.0
"""25%""",null,null,null,2795.0,null,null,null,null,null,null,null
"""50%""",null,null,null,138840.0,null,null,null,null,null,null,null
"""75%""",null,null,null,218502.0,null,null,null,null,null,null,null
"""max""","""7zwWMJUV62QNNLz15hrWZl""","""2024-12-15 23:06:25""","""windows""",1.561125e6,"""🪐""","""落日飛車 Sunset Rollercoaster""","""母親""","""unknown""","""unknown""",1.0,1.0


In [8]:
spotify_history_df.collect_schema()

Schema([('spotify_track_uri', String),
        ('ts', String),
        ('platform', String),
        ('ms_played', Int64),
        ('track_name', String),
        ('artist_name', String),
        ('album_name', String),
        ('reason_start', String),
        ('reason_end', String),
        ('shuffle', Boolean),
        ('skipped', Boolean)])

In [11]:
spotify_history_df.estimated_size('mb')

17.740535736083984

In [19]:
spotify_history_df = spotify_history_df.with_columns(
    pl.col('reason_start').fill_null('unknown'),
    pl.col('reason_end').fill_null('unknown'),
    pl.col('ms_played').cast(pl.UInt32),
    pl.col('ts').str.strptime(pl.Datetime(), format='%Y-%m-%d %H:%M:%S')
)

In [21]:
spotify_data_dictionary_description_df.collect_schema()

Schema([('Field', String), ('Description', String)])

In [22]:
spotify_data_dictionary_description_df.describe()

statistic,Field,Description
str,str,str
"""count""","""11""","""11"""
"""null_count""","""0""","""0"""
"""mean""",null,null
"""std""",null,null
"""min""","""album_name""","""Name of the album"""
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""","""ts""","""Why the track started"""


In [23]:
spotify_data_dictionary_description_df

Field,Description
str,str
"""spotify_track_uri""","""Spotify URI that uniquely iden…"
"""ts""","""Timestamp indicating when the …"
"""platform""","""Platform used when streaming t…"
"""ms_played""","""Number of milliseconds the str…"
"""track_name""","""Name of the track"""
…,…
"""album_name""","""Name of the album"""
"""reason_start""","""Why the track started"""
"""reason_end""","""Why the track ended"""


In [27]:
spotify_history_df.write_parquet(
    r'D:\Datasets\Data Werehouse\Top Spotify Listening History Songs in Countries\spotify_history.parquet',
    compression='zstd',
    compression_level=22,
)

In [28]:
spotify_data_dictionary_description_df.write_parquet(
    r'D:\Datasets\Data Werehouse\Top Spotify Listening History Songs in Countries\spotify_data_dictionary_description'
    r'.parquet',
    compression='zstd',
    compression_level=22,
)

# Data analysis